In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from kerastuner import Hyperband, HyperModel
from tensorflow.keras import layers

In [2]:
dataset = pd.read_csv('data/winequality/winequality-red.csv', delimiter=';')
dataset = dataset.dropna()

train_dataset = dataset.sample(frac=0.8, random_state=0)
val_test_dataset = dataset.drop(train_dataset.index)
val_dataset = val_test_dataset.sample(frac=0.5, random_state=0)
test_dataset = val_test_dataset.drop(val_dataset.index)

train_features = train_dataset.copy()
val_features = val_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('quality')
val_labels = val_features.pop('quality')
test_labels = test_features.pop('quality')

In [3]:
min_value = 1
max_value = 8
step_size = 1

class DeepHyperModel(HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential([
            layers.experimental.preprocessing.Normalization(),
            layers.Dense(units=hp.Int(
                'units_1',
                min_value=min_value,
                max_value=max_value,
                step=step_size
            ), activation='relu'),
            layers.Dense(units=hp.Int(
                'units_2',
                min_value=min_value,
                max_value=max_value,
                step=step_size
            ), activation='relu'),
            layers.Dense(units=hp.Int(
                'units_3',
                min_value=min_value,
                max_value=max_value,
                step=step_size
            ), activation='relu'),
            layers.Dense(units=hp.Int(
                'units_4',
                min_value=min_value,
                max_value=max_value,
                step=step_size
            ), activation='relu'),
            layers.Dense(units=hp.Int(
                'units_5',
                min_value=min_value,
                max_value=max_value,
                step=step_size
            ), activation='relu'),
            layers.Dense(units=hp.Int(
                'units_6',
                min_value=min_value,
                max_value=max_value,
                step=step_size
            ), activation='relu'),
            layers.Dense(units=hp.Int(
                'units_7',
                min_value=min_value,
                max_value=max_value,
                step=step_size
            ), activation='relu'),
            layers.Dense(units=hp.Int(
                'units_8',
                min_value=min_value,
                max_value=max_value,
                step=step_size
            ), activation='relu'),
            layers.Dense(1)
        ])

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float(
                'learning_rate',
                min_value=1e-6,
                max_value=0.01,
                sampling='LOG'
            )),
            loss='mean_squared_error'
        )
        return model

In [4]:
HYPERBAND_MAX_EPOCHS = 200
EXECUTION_PER_TRIAL = 3

hypermodel = DeepHyperModel()

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_loss',
    seed=42,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='deep_net'
)

tuner.search_space_summary()

Search space summary
Default search space size: 9
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
units_6 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
units_7 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
units_8 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': None}
learning_rate (Float)
{'default': 1e-06, 'conditions': [], 'mi

In [ ]:
tuner.search(train_features, train_labels, validation_data=(val_features, val_labels))

Trial 182 Complete [00h 00m 03s]
val_loss: 54.77245966593424

Best val_loss So Far: 0.4280596176783244
Total elapsed time: 00h 10m 36s

Search: Running Trial #183

Hyperparameter    |Value             |Best Value So Far 
units_1           |5                 |2                 
units_2           |4                 |7                 
units_3           |5                 |2                 
units_4           |3                 |5                 
units_5           |8                 |2                 
units_6           |7                 |4                 
units_7           |5                 |5                 
units_8           |2                 |4                 
learning_rate     |1.2722e-06        |0.007258          
tuner/epochs      |8                 |67                
tuner/initial_e...|0                 |23                
tuner/bracket     |3                 |4                 
tuner/round       |0                 |3                 

Epoch 1/8
40/40 [====================

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_features, train_labels, validation_data=(val_features, val_labels), epochs=200)

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['loss'], label='training data')
plt.plot(history.history['val_loss'], label='validation data')
plt.title('Losses')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
loss = best_model.evaluate(val_features, val_labels)
print(loss)

loss = best_model.evaluate(test_features, test_labels)
print(loss)

In [ ]:
predicted_test_labels = model.predict(test_features)
print(np.column_stack([predicted_test_labels, test_labels]))